# Linear regression

In [1]:
# Notebook setup

# Imports
import itertools
import math

from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from patsy import dmatrices
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as subplots
from sklearn import linear_model, metrics
import statsmodels.api as sm
from statsmodels.graphics.gofplots import qqplot

# Set the random seed for reproducability
np.random.seed(1337)

### Problem setting

Given a data set $\{y_i, x_{i1}, x_{i2}, ..., x_{ip}\}_{i=1}^n$ a linear regression model assumes that the relationship between the dependent variable $y$ and the vector of regressors $\bold{x}$ is linear.

This relationship is modeled through a disturbance term $\epsilon$, an unobserved random variable that adds "noise" to the linear relationship between the dependent variable and regressors.

The model takes the following form:

$y_i = \beta_0 + \beta_1 x_{i1} + ... + \beta_p x_{ip} + \epsilon_i = \bold x_i^T \bold\beta + \epsilon_i$ where $(i=1,..., n)$

Using matrix notation this can be rewrtitten the following way:

$\bold{y = X \beta + \epsilon}$

Where:

$\bold y = \begin{bmatrix} y_1 \\ y_2 \\ \vdots \\ y_n \end{bmatrix}$ are the observed values AKA the endogenous variable,

$\bold X = \begin{bmatrix} 1 & x_{11} & ... & x_{1p} \\ 1 & x_{21} & ... & x_{2p} \\ \vdots & \vdots & & \vdots \\ 1 & x_{n1} & ... & x_{np} \end{bmatrix}$ are the regressors AKA exogenous variables,

$\bold \beta = \begin{bmatrix} \beta_0 \\ \beta_1 \\ \beta_2 \\ \vdots \\ \beta_p \end{bmatrix}$ is the parameter vector, AKA the regression coefficients where $\beta_0$ is the intercept term,

$\bold \epsilon = \begin{bmatrix} \epsilon_1 \\ \epsilon_2 \\ \vdots \\ \epsilon_n \end{bmatrix}$ is the error term AKA noise or disturbance term.

Linear regerssion modelling is possible if the following assumptions hold true for the data:

1. The predictor variables can be treated as error free
2. The the endogenous variable is a linear combination of the exogenous variables and the coefficients
3. The variance of the errors does not depend on the exogenous variables and can be treated as constant throughout the data set
4. The errors are independent of each other
5. There must not be a perfect multicollinearity between the columns of the exogenous variables

### Solution

The simplest solution is the OLS (ordinary least squares) coefficient estimates:

$\hat \beta = ( \bold X' \bold X) ^{-1} \bold X'y$

### Evaluation

A list of metrics that can be used to evaluate a set of coefficients where an estimate of the coefficients $\hat \beta$ and an estimate of the exogenous variables with the coefficient estimate $\hat y = \bold X \hat \beta$.

Residual sum of squares: $RSS = \sum \limits_{i=1}^{n} ( \hat y_i - \bar y )^2$

Total sum of squares: $TSS = \sum \limits_{i=1}^{n} ( y_i - \bar y )^2$

Explained sum of squares: $ESS = TSS - RSS$

$R^2$ score: $R^2 = 1 - \frac{RSS}{TSS}$

Mean absolute error: $MAE = \frac{1}{n} \sum \limits_{i=1}^{n} | \hat y_i - y_i |$

Mean squared error: $MSE = \frac{1}{n} \sum \limits_{i=1}^{n} | \hat y_i - y_i | ^ 2$

Explained variance score: $1 - \frac{Var(y - \hat y)}{Var(y)}$


### Linear regression on the King county house sales data set

In [2]:
# Load the data set and do some very minimal feature engineering

d = pd.read_csv(
    "../data/king_county_house_sales/kc_house_data.csv", parse_dates=["date"]
)
del d["id"]
del d["date"]
del d["lat"]
del d["long"]
d["age_built"] = 2015 - d.yr_built
del d["yr_built"]
d["past_30yr_renovated"] = (d.yr_renovated >= 1985).astype(int)
del d["yr_renovated"]
print(d.info())
display(d.describe().T)
display(d.sample(10).T)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   price                21613 non-null  float64
 1   bedrooms             21613 non-null  int64  
 2   bathrooms            21613 non-null  float64
 3   sqft_living          21613 non-null  int64  
 4   sqft_lot             21613 non-null  int64  
 5   floors               21613 non-null  float64
 6   waterfront           21613 non-null  int64  
 7   view                 21613 non-null  int64  
 8   condition            21613 non-null  int64  
 9   grade                21613 non-null  int64  
 10  sqft_above           21613 non-null  int64  
 11  sqft_basement        21613 non-null  int64  
 12  zipcode              21613 non-null  int64  
 13  sqft_living15        21613 non-null  int64  
 14  sqft_lot15           21613 non-null  int64  
 15  age_built            21613 non-null 

,count,mean,std,min,25%,50%,75%,max
price,21613.0,540088.141767,367127.196483,75000.0,321950.00,450000.00,645000.0,7700000.0
bedrooms,21613.0,3.370842,0.930062,0.0,3.00,3.00,4.0,33.0
bathrooms,21613.0,2.114757,0.770163,0.0,1.75,2.25,2.5,8.0
sqft_living,21613.0,2079.899736,918.440897,290.0,1427.00,1910.00,2550.0,13540.0
sqft_lot,21613.0,15106.967566,41420.511515,520.0,5040.00,7618.00,10688.0,1651359.0
floors,21613.0,1.494309,0.539989,1.0,1.00,1.50,2.0,3.5
waterfront,21613.0,0.007542,0.086517,0.0,0.00,0.00,0.0,1.0
view,21613.0,0.234303,0.766318,0.0,0.00,0.00,0.0,4.0
condition,21613.0,3.409430,0.650743,1.0,3.00,3.00,4.0,5.0
grade,21613.0,7.656873,1.175459,1.0,7.00,7.00,8.0,13.0


,1455,14997,18667,5682,11815,20178,17978,4475,12797,20441
price,382000.00,731688.0,526500.0,286000.0,467000.0,551000.00,615000.00,670000.0,395000.0,2998000.0
bedrooms,3.00,4.0,3.0,3.0,5.0,5.00,3.00,3.0,1.0,5.0
bathrooms,2.25,3.0,1.5,2.5,2.0,3.75,3.25,3.0,1.0,4.0
sqft_living,1800.00,2630.0,1310.0,1680.0,2080.0,3090.00,1470.00,2980.0,790.0,6670.0
sqft_lot,4500.00,5772.0,7236.0,5000.0,4000.0,4943.00,1152.00,3730.0,3000.0,16481.0
floors,2.00,2.0,1.0,2.0,1.0,2.00,3.00,2.0,1.0,2.0
waterfront,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0
view,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0
condition,3.00,3.0,4.0,3.0,4.0,3.00,3.00,3.0,3.0,3.0
grade,7.00,9.0,7.0,8.0,6.0,10.00,8.00,9.0,6.0,12.0


In [3]:
# Distribution of a selected set of numeric variables

d_plot = d
cols_names = [
    "price",
    "bedrooms",
    "bathrooms",
    "sqft_living",
    "floors",
    "view",
    "condition",
    "grade",
    "age_built",
    "past_30yr_renovated",
]
n_plots = len(cols_names)
n_plot_cols = 2
n_plot_rows = math.ceil(n_plots / n_plot_cols)

fig = subplots.make_subplots(
    rows=n_plot_rows,
    cols=n_plot_cols,
    subplot_titles=cols_names,
    horizontal_spacing=0.05,
    vertical_spacing=0.28 - 0.05 * n_plot_rows,
)
for i, val_name in enumerate(cols_names):
    fig.add_trace(
        go.Histogram(
            x=None if d_plot[val_name].dtype == "object" else d_plot[val_name],
            y=d_plot[val_name] if d_plot[val_name].dtype == "object" else None,
            name="",
            nbinsx=None,
            xbins=dict(size=None, start=None, end=None),
            marker=dict(color="cornflowerblue"),
        ),
        row=1 + i // n_plot_cols,
        col=1 + i % n_plot_cols,
    )
    fig.update_xaxes(title_text="", row=1 + i // n_plot_cols, col=1 + i % n_plot_cols)
    fig.update_yaxes(title_text="", row=1 + i // n_plot_cols, col=1 + i % n_plot_cols)
if d_plot[val_name].dtype == "object":
    fig.update_traces(hovertemplate="value: %{y}<br>count: %{x}")
else:
    fig.update_traces(hovertemplate="value: %{x}<br>count: %{y}")
fig.update_layout(
    template="plotly_white",
    width=1200,
    height=80 + 250 * n_plot_rows,
    title="Distributions",
    legend_title="Legend",
    showlegend=False,
)

In [4]:
# Feature correlations

d_plot = d
col_names = d.columns
corr = np.flip(np.corrcoef(d_plot[col_names], rowvar=False), 0)
fig = go.Figure()
fig.add_trace(
    go.Heatmap(
        x=col_names,
        y=col_names[::-1],
        z=corr,
        zmin=-1,
        zmax=1,
        zauto=False,
        name="",
        colorscale="RdBu_r",
        showscale=False,
        text=corr,
        texttemplate="%{text:0.2f}",
        textfont={"size": 12},
    )
)
fig.update_traces(hovertemplate="x: %{x}" + "<br>y: %{y}" + "<br>value: %{z:0.3f}")
fig.update_layout(
    template="plotly_white",
    width=200 + 50 * corr.shape[0],
    height=200 + 50 * corr.shape[0],
    title="Correlations",
)
fig.update_xaxes(tickangle=270)
fig.show()

In [5]:
# Correlation plot of a selected set of variables

d_plot = d.sample(300)
col_names = [
    "price", "bedrooms", "sqft_living", "grade", "view", "age_built"
]
pairs = list(itertools.combinations(col_names, r=2))
n_plots = len(pairs)
n_plot_cols = 4
n_plot_rows = math.ceil(n_plots / n_plot_cols)

fig = subplots.make_subplots(
    rows=n_plot_rows,
    cols=n_plot_cols,
    subplot_titles=[
        f"correlation: {np.corrcoef(d[pair[0]], d[pair[1]])[0, 1]:0.2f}"
        for pair in pairs
    ],
    horizontal_spacing=0.08,
    vertical_spacing=0.28 - 0.05 * n_plot_rows,
)
for i in range(n_plots):
    fig.add_trace(
        go.Scatter(
            x=d_plot[pairs[i][0]],
            y=d_plot[pairs[i][1]],
            name="",
            mode="markers",
            marker=dict(color="cornflowerblue", size=4, symbol="circle"),
        ),
        row=1 + i // n_plot_cols,
        col=1 + i % n_plot_cols,
    )
    fig.update_xaxes(
        title_text=pairs[i][0], row=1 + i // n_plot_cols, col=1 + i % n_plot_cols
    )
    fig.update_yaxes(
        title_text=pairs[i][1], row=1 + i // n_plot_cols, col=1 + i % n_plot_cols
    )
fig.update_traces(hovertemplate="x: %{x:0.3f}" + "<br>y: %{y:0.3f}")
fig.update_layout(
    template="plotly_white",
    width=1200,
    height=80 + 280 * n_plot_rows,
    title="Correlations",
    legend_title="Legend",
    showlegend=False,
)

In [6]:
# OLS regression with a selected set of variables

exog_vars = ["bedrooms", "sqft_living", "view", "grade", "past_30yr_renovated"]

y, X = dmatrices("price ~ " + " + ".join(exog_vars), data=d, return_type="dataframe")

model = sm.OLS(y, X)
res = model.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.584
Model:                            OLS   Adj. R-squared:                  0.584
Method:                 Least Squares   F-statistic:                     6057.
Date:                Wed, 07 Feb 2024   Prob (F-statistic):               0.00
Time:                        15:02:32   Log-Likelihood:            -2.9814e+05
No. Observations:               21613   AIC:                         5.963e+05
Df Residuals:                   21607   BIC:                         5.963e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept           -4.523e+05   1

### Future work

   * Plot partial regressions with one variable
   * Residual plot
   * Leverage
   * Scale-location plots
   * Outliers and Cook's distance
   * Feature selection
   * Multicollinearity and confounding variables
   * Generalised linear models